In [159]:
import io
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

from os.path import join

import os
from tqdm import tqdm

def safe_mkdir(path):
    """ Create a directory if there isn't one already. """
    try:
        os.makedirs(path)
    except OSError:
        pass

In [160]:
dir_crop = glob("hdr-dsp-real-dataset/crop/*")
dir_crop.sort()

dir_ratio = glob("hdr-dsp-real-dataset/ratios/*")
dir_ratio.sort()

dir_sat = glob("hdr-dsp-real-dataset/satmask/*")
dir_sat.sort()

In [161]:
len(dir_crop)

2660

In [162]:
ssc13index = []
for i,crop in enumerate(dir_crop):
    if "ssc13" in crop:
        ssc13index.append(i)

In [163]:
len(ssc13index)

105

In [165]:
TrainData = {}
TrainRatios = {}

ValData = {}
ValRatios = {}

TestData = {}
TestRatios = {}

for i in range(3, 16):
    TrainData[str(i)] = []
    TrainRatios[str(i)] = []
    
    ValData[str(i)] = []
    ValRatios[str(i)] = []
    
    TestData[str(i)] = []
    TestRatios[str(i)] = []

    
    
for i in tqdm(range(2500)):
    data = np.load(dir_crop[i])
    ratio = np.load(dir_ratio[i])
    sat = np.load(dir_sat[i])
    
    nbFrames = len(ratio)
    
    nonSatIndex = list(range(nbFrames))
    
    if i not in ssc13index:
        for j in range(nbFrames):
            if False in sat[j]:
                nonSatIndex.remove(j)
    else:
        for j in range(nbFrames):
            if data[j].max() > 2600:
                nonSatIndex.remove(j)
                
    if len(nonSatIndex) >= 3:
        if i < 2000:
            TrainData[str(len(nonSatIndex))].append(data[nonSatIndex])
            TrainRatios[str(len(nonSatIndex))].append(ratio[nonSatIndex])
        elif i <2200:
            ValData[str(len(nonSatIndex))].append(data[nonSatIndex])
            ValRatios[str(len(nonSatIndex))].append(ratio[nonSatIndex])
        else:
            TestData[str(len(nonSatIndex))].append(data[nonSatIndex])
            TestRatios[str(len(nonSatIndex))].append(ratio[nonSatIndex])


for j in range(3, 16):
    if len(TrainRatios[str(j)]) > 0:
        folder = "SkySat_ME_noSaturation/train/{}".format(j)
        safe_mkdir(folder)
        np.save(join(folder, "trainLR.npy"), np.array(TrainData[str(j)]))
        np.save(join(folder, "trainRatio.npy"), np.array(TrainRatios[str(j)]))

    if len(ValRatios[str(j)]) > 0:
        folder = "SkySat_ME_noSaturation/val/{}".format(j)
        safe_mkdir(folder)
        np.save(join(folder, "valLR.npy"), np.array(ValData[str(j)]))
        np.save(join(folder, "valRatio.npy"), np.array(ValRatios[str(j)]))
        
    if len(TestRatios[str(j)]) > 0:
        folder = "SkySat_ME_noSaturation/test/{}".format(j)
        safe_mkdir(folder)
        np.save(join(folder, "testLR.npy"), np.array(TestData[str(j)]))
        np.save(join(folder, "testRatio.npy"), np.array(TestRatios[str(j)]))
    
    

100%|█████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:03<00:00, 657.64it/s]
